<a href="https://colab.research.google.com/github/yuyoungleee/document-qa/blob/main/Gemini_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from IPython.display import Markdown

import os
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"

In [ ]:
%pip install -U --quiet langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("네이버에 대해 보고서를 작성해줘")
Markdown(result.content)

In [ ]:
%pip install -U --quiet langchain tiktoken pypdf sentence_transformers chromadb

In [ ]:
# from langchain.chains import RetrievalQA
# from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

In [ ]:
 retriever = docsearch.as_retriever(
                                    search_type="mmr",
                                    search_kwargs={'k':3, 'fetch_k': 10})
 retriever.get_relevant_documents("혁신성장 정책금융에 대해서 설명해줘")

[Document(page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로\n  CIS이슈리포트 2022-2 호 | 3 |1. 들어가며\n▶혁신성장 정책금융기관은 건강한 혁신산업 생태계를 조성하기 위해 기업 성장에 필요한 자금을 \n지원하는 혁신성장 정책금융 제도를 시행하고 있음\n￮혁신성장 정책금융기관은 혁신성장에 대한 정의를 구체화한 정책금융 가이드라인*에 따라 혁신성장  \n산업육성을 위한 정책금융 업무를 추진 중임\n       * 혁신성장 기업발굴 및 금융지원을 위해 활용하는 기준으로 , ‘9대 테마-46개 분야-296개 품목’으로 구성\n▶혁신성장 정책금융 제도 시행 이후 공급 규모가 매년 증가하는 등, 미래 혁신성장 분야의 글로벌 \n경쟁력 확보를 위한 금융지원이 지속 추진 중임\n￮정책금융기관의 혁신성장 분야 정책금융 공급규모는 2017년 240,787 억 원에서  연평균  37.2% 증가\n하여 2021 년 854,338 억 원에 이르는 등 그 외연을 확장해나가고 있음', metadata={'page': 2, 'source': '/content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf'}),
 Document(page_content='￮또한, 미래 성장성은 유망하나 단기 수익 창출이 어려운 산업의 지원 강화를 위해 정책금융 뿐만 아니라  \n민관주도의 역동적 금융으로 혁신성장 금융지원 영역을 확대할 필요가 있음', metadata={'page': 1, 'source': '/content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf'}),
 Document(page_content='동 품목으로의 정책금융 공급은 지속 증가할 것으로 전망됨\n▶센서측정 테마의 경우 정보통신 , 전기전자 테마 대비 정책금융 공급 규모는 작

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap

template = """Answer the question as based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
gemini = ChatGoogleGenerativeAI(model="gemini-pro", temperature = 0)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question": lambda x: x['question']
}) | prompt | gemini

In [ ]:
Markdown(chain.invoke({'question': "혁신성장 정책금융에 대해서 설명해줘"}).content)

혁신성장 정책금융은 건강한 혁신산업 생태계를 조성하기 위해 기업 성장에 필요한 자금을 지원하는 혁신성장 정책금융 제도입니다. 혁신성장 정책금융기관은 혁신성장에 대한 정의를 구체화한 정책금융 가이드라인에 따라 혁신성장 산업육성을 위한 정책금융 업무를 추진하고 있습니다. 혁신성장 정책금융 제도 시행 이후 공급 규모가 매년 증가하는 등, 미래 혁신성장 분야의 글로벌 경쟁력 확보를 위한 금융지원이 지속 추진 중입니다.